In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.svm import SVR

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

import mlflow

In [3]:
# set the dagshub tracking server

mlflow.set_tracking_uri("https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow")

In [4]:
import dagshub
dagshub.init(repo_owner='ThE-GuY-sHuBhAm', repo_name='uber-demand-prediction', mlflow=True)

Accessing as ThE-GuY-sHuBhAm

Initialized MLflow to track repo "ThE-GuY-sHuBhAm/uber-demand-prediction"

Repository ThE-GuY-sHuBhAm/uber-demand-prediction initialized!

In [5]:
# load the training and test data

train_data_path = "../data/processed/train.csv"
test_data_path = "../data/processed/test.csv"

train_df = pd.read_csv(train_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

test_df = pd.read_csv(test_data_path, parse_dates=["tpep_pickup_datetime"]).set_index("tpep_pickup_datetime")

train_df

,lag_1,lag_2,lag_3,lag_4,region,total_pickups,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,187,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,194,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,180,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,197,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185,185.0,4
...,...,...,...,...,...,...,...,...
2016-02-29 22:45:00,15.0,9.0,11.0,11.0,29,12,12.0,0
2016-02-29 23:00:00,12.0,15.0,9.0,11.0,29,17,14.0,0
2016-02-29 23:15:00,17.0,12.0,15.0,9.0,29,15,14.0,0


In [6]:
# missing value in training data

train_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [7]:
# missing values in the test data

test_df.isna().sum()

lag_1            0
lag_2            0
lag_3            0
lag_4            0
region           0
total_pickups    0
avg_pickups      0
day_of_week      0
dtype: int64

In [8]:
# make X_train and y_train

X_train = train_df.drop(columns=["total_pickups"])

y_train = train_df["total_pickups"]

In [9]:
X_train.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-01-01 01:00:00,160.0,149.0,120.0,58.0,0,161.0,4
2016-01-01 01:15:00,187.0,160.0,149.0,120.0,0,175.0,4
2016-01-01 01:30:00,194.0,187.0,160.0,149.0,0,177.0,4
2016-01-01 01:45:00,180.0,194.0,187.0,160.0,0,185.0,4
2016-01-01 02:00:00,197.0,180.0,194.0,187.0,0,185.0,4


In [10]:
# make X_test and y_test

X_test = test_df.drop(columns=["total_pickups"])

y_test = test_df["total_pickups"]

In [11]:
X_test.head()

,lag_1,lag_2,lag_3,lag_4,region,avg_pickups,day_of_week
tpep_pickup_datetime,,,,,,,
2016-03-01 00:00:00,36.0,44.0,31.0,29.0,0,39.0,1
2016-03-01 00:15:00,41.0,36.0,44.0,31.0,0,37.0,1
2016-03-01 00:30:00,35.0,41.0,36.0,44.0,0,41.0,1
2016-03-01 00:45:00,47.0,35.0,41.0,36.0,0,38.0,1
2016-03-01 01:00:00,34.0,47.0,35.0,41.0,0,35.0,1


In [12]:
from sklearn import set_config

set_config(transform_output="pandas")

In [13]:
# encode the data

encoder = ColumnTransformer([
    ("ohe", OneHotEncoder(drop="first",sparse_output=False), ["region","day_of_week"])
], remainder="passthrough", n_jobs=-1,force_int_remainder_cols=False)

In [14]:
encoder

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('ohe', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'passthrough'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",-1
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. ``""{featu

In [15]:
# encode the train and test data

X_train_encoded = encoder.fit_transform(X_train)
X_test_encoded = encoder.transform(X_test)

c:\Users\Lenovo\Desktop\Projects\uber-demand-prediction\uber_env\Lib\site-packages\sklearn\compose\_column_transformer.py:978: FutureWarning: The parameter `force_int_remainder_cols` is deprecated and will be removed in 1.9. It has no effect. Leave it to its default value to avoid this warning.
  warnings.warn(


In [16]:
import optuna
import tqdm 

In [17]:
# set the experiment

mlflow.set_experiment("Model Selection")

<Experiment: artifact_location='mlflow-artifacts:/137bcf9dcddf45f785d5b013e9d40a99', creation_time=1770806605350, experiment_id='0', last_update_time=1770806605350, lifecycle_stage='active', name='Model Selection', tags={}>

In [18]:
def objective(trial):
    # start the child run
    with mlflow.start_run(nested=True) as child:
        
        # model name search space
        list_of_models = ["LR", "RF", "GBR", "XGBR"]
        model_name = trial.suggest_categorical("model_name", list_of_models)
    
        if model_name == "LR":
            model = LinearRegression()
    
        elif model_name == "RF":
            n_estimators_rf = trial.suggest_int("n_estimators_rf",10,100,step=10)
            max_depth_rf = trial.suggest_int("max_depth_rf",3,10)
            model = RandomForestRegressor(n_estimators=n_estimators_rf, 
                                          max_depth=max_depth_rf, 
                                          random_state=42, n_jobs=-1)
    
        elif model_name == "GBR":
            n_estimators_gb = trial.suggest_int("n_estimators_gb",10,100,step=10)
            learning_rate_gb = trial.suggest_float("learning_rate_gb",1e-4,1e-1, log=True)
            model = GradientBoostingRegressor(n_estimators=n_estimators_gb, 
                                              learning_rate=learning_rate_gb,
                                             random_state=42)
    
        elif model_name == "XGBR":
            n_estimators_xgb = trial.suggest_int("n_estimators_xgb",10,100,step=10)
            learning_rate_xgb = trial.suggest_float("learning_rate_xgb",1e-4,1e-1, log=True)
            max_depth_xgb = trial.suggest_int("max_depth_xgb",3,10)
            model = XGBRegressor(n_estimators=n_estimators_xgb,
                                learning_rate=learning_rate_xgb,
                                max_depth=max_depth_xgb)
    
        # log the model name
        mlflow.log_param("model_name",model_name)
        
        # log the model parameters
        mlflow.log_params(model.get_params())
        
        # fit on the data
        model.fit(X_train_encoded,y_train)
    
        # get the predictions
        y_pred = model.predict(X_test_encoded)
    
        # calculate the loss
        loss = mean_absolute_percentage_error(y_test, y_pred)
    
        # log the metric
        mlflow.log_metric("MAPE",loss)
        return loss

In [19]:
# optimize the objective function

with mlflow.start_run(run_name="best_model", nested=True) as parent:

    # create a study object
    study = optuna.create_study(study_name="model_selection", direction="minimize")
    # optimize the objective function
    study.optimize(func=objective, n_trials=50, n_jobs=-1)
    
    # log the best parameters
    mlflow.log_params(study.best_params)
    # log the best error value
    mlflow.log_metric("Best_MAPE", study.best_value)

[I 2026-02-11 18:06:57,314] A new study created in memory with name: model_selection


🏃 View run popular-wren-45 at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0/runs/c5ce78c3aed24752bc9c62ed5d20ced2
🧪 View experiment at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0
🏃 View run receptive-colt-917 at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0/runs/535db377ca554ab5bdf8529d34b2e3c4
🧪 View experiment at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0


[I 2026-02-11 18:07:39,635] Trial 18 finished with value: 0.5187374163731608 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 50, 'learning_rate_gb': 0.06180825154774774}. Best is trial 18 with value: 0.5187374163731608.


🏃 View run aged-vole-777 at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0/runs/497a9140b8d34607826f4497195a7177
🧪 View experiment at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0


[I 2026-02-11 18:07:45,622] Trial 15 finished with value: 6.462602138519287 and parameters: {'model_name': 'XGBR', 'n_estimators_xgb': 80, 'learning_rate_xgb': 0.00022340319019103726, 'max_depth_xgb': 7}. Best is trial 18 with value: 0.5187374163731608.


...

🏃 View run bold-squid-514 at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0/runs/018f1abbf7714ae58c89ccac86ad8038
🧪 View experiment at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0


[I 2026-02-11 18:12:09,003] Trial 19 finished with value: 6.469269924514753 and parameters: {'model_name': 'GBR', 'n_estimators_gb': 80, 'learning_rate_gb': 0.00021992033093034015}. Best is trial 17 with value: 0.07934790285463075.


🏃 View run bemused-ox-256 at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0/runs/42e480140b7b41d8b35686090f9f056f
🧪 View experiment at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0


[I 2026-02-11 18:12:30,920] Trial 0 finished with value: 0.1582508899731008 and parameters: {'model_name': 'RF', 'n_estimators_rf': 60, 'max_depth_rf': 8}. Best is trial 17 with value: 0.07934790285463075.


🏃 View run best_model at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0/runs/22f736b679e2499e91c74f595fb1c2f1
🧪 View experiment at: https://dagshub.com/ThE-GuY-sHuBhAm/uber-demand-prediction.mlflow/#/experiments/0


In [20]:
# best value

study.best_value

0.07934790285463075

In [21]:
# best parameters

study.best_params

{'model_name': 'LR'}

In [22]:
# model value counts

study.trials_dataframe()['params_model_name'].value_counts()

params_model_name
LR      28
RF       8
GBR      8
XGBR     6
Name: count, dtype: int64

In [23]:
from optuna.visualization import (
    plot_optimization_history, 
    plot_parallel_coordinate, 
    plot_param_importances
)

In [24]:
import plotly
plot_optimization_history(study)

In [25]:
plot_parallel_coordinate(study, params=["model_name"])

In [26]:
# train the linear regression model

lr = LinearRegression()

lr.fit(X_train_encoded, y_train)

# get predictions
y_pred_train = lr.predict(X_train_encoded) 
y_pred_test = lr.predict(X_test_encoded)

# loss

mape_train = mean_absolute_percentage_error(y_train, y_pred_train)
mape_test = mean_absolute_percentage_error(y_test, y_pred_test)

print("The training error is ", mape_train)
print("The test error is ", mape_test)

The training error is  0.08778013304566512
The test error is  0.07934790285463075


In [27]:
lr.coef_

array([-2.33737604,  0.71512405, -0.55601505, -1.25311068, -3.20463231,
       -0.86685973, -2.79925402, -3.62516859,  0.41386463, -2.9376376 ,
       -1.97624678, -3.75050442,  0.51806283, -2.54033388, -2.43297463,
        0.47632075,  0.61254786, -4.7417372 , -2.03077217, -1.26960984,
       -4.03690273, -2.08863167, -1.0414428 ,  0.73561736, -0.99999442,
       -0.85944985, -2.43098478,  0.67112238,  0.57385071, -0.11719951,
       -0.28045898, -0.37180749, -0.5238324 , -0.4233113 , -0.34045774,
       -0.54170892, -0.36264553, -0.2493965 , -0.31905518,  2.4912456 ])

In [28]:
def tune_ridge(trial):
    # hyperparameter space
    alpha = trial.suggest_float("alpha",30,100)
    
    # make the model object
    ridge = Ridge(alpha=alpha, random_state=42)
    
    # train the model
    ridge.fit(X_train_encoded, y_train)
    
    # get predictions
    y_pred = ridge.predict(X_test_encoded)
    
    # calculate loss
    loss = mean_absolute_percentage_error(y_test, y_pred)

    return loss
        

In [29]:
# create study

study = optuna.create_study(study_name="tune_model", direction="minimize")

[I 2026-02-11 18:12:37,505] A new study created in memory with name: tune_model


In [30]:
# optimize

study.optimize(func=tune_ridge, n_trials=100, n_jobs=-1, show_progress_bar=True)

  0%|          | 0/100 [00:00<?, ?it/s]

[I 2026-02-11 18:12:37,804] Trial 0 finished with value: 0.0792521809290436 and parameters: {'alpha': 41.954253172773086}. Best is trial 0 with value: 0.0792521809290436.
[I 2026-02-11 18:12:37,947] Trial 1 finished with value: 0.07924698623710608 and parameters: {'alpha': 44.725186011995824}. Best is trial 1 with value: 0.07924698623710608.
[I 2026-02-11 18:12:38,049] Trial 2 finished with value: 0.07923842145837352 and parameters: {'alpha': 49.50583388315353}. Best is trial 2 with value: 0.07923842145837352.
[I 2026-02-11 18:12:38,064] Trial 3 finished with value: 0.07920420598593327 and parameters: {'alpha': 70.72619849474977}. Best is trial 3 with value: 0.07920420598593327.
[I 2026-02-11 18:12:38,093] Trial 6 finished with value: 0.07920115861265495 and parameters: {'alpha': 72.77856425523696}. Best is trial 6 with value: 0.07920115861265495.
[I 2026-02-11 18:12:38,099] Trial 5 finished with value: 0.07926568653661009 and parameters: {'alpha': 34.963351956797425}. Best is trial 6 

In [31]:
# best parameters

study.best_params

{'alpha': 99.9967768354129}

In [32]:
# best value

study.best_value

0.07916406951717872